In [275]:
import pandas as pd
file_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\3rd Sem\NLP\Assignment_2\data.csv"  # Replace 'path_to_your_file.csv' with the actual path to your CSV file

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
print(df.Sentence)
print(df.Sentiment)


0       The GeoSolutions technology will leverage Bene...
1       $ESI on lows, down $1.50 to $2.50 BK a real po...
2       For the last quarter of 2010 , Componenta 's n...
3       According to the Finnish-Russian Chamber of Co...
4       The Swedish buyout firm has sold its remaining...
                              ...                        
5837    RISING costs have forced packaging producer Hu...
5838    Nordic Walking was first used as a summer trai...
5839    According shipping company Viking Line , the E...
5840    In the building and home improvement trade , s...
5841    HELSINKI AFX - KCI Konecranes said it has won ...
Name: Sentence, Length: 5842, dtype: object
0       positive
1       negative
2       positive
3        neutral
4        neutral
          ...   
5837    negative
5838     neutral
5839     neutral
5840     neutral
5841    positive
Name: Sentiment, Length: 5842, dtype: object


In [ ]:
df.Sentiment.unique

In [ ]:
df.loc[df["Sentiment"] == "negative", "Sentiment"] = -1
df.loc[df["Sentiment"] == "neutral", "Sentiment"] = 0
df.loc[df["Sentiment"] == "positive", "Sentiment"] = 1

In [ ]:
%pip install keras.preprocessing

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from gensim.models import Word2Vec

import numpy as np
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import string



# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


In [ ]:

# Tokenize sentences, remove stopwords, and handle punctuation
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    return tokens

# Preprocess sentences
preprocessed_sentences = [preprocess_text(sentence) for sentence in df.Sentence]

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_sentences, df.Sentiment, test_size=0.10, random_state=1)
print("Sample preprocessed data:")
print(preprocessed_sentences[0])


In [ ]:

# CBOW Model
cbow_model = Word2Vec(sentences=X_train, vector_size=100, window=2, sg=0, min_count=1, workers=4)

# Skip-gram Model
skipgram_model = Word2Vec(sentences=X_train, vector_size=100, window=2, sg=1, min_count=1, workers=4)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

In [ ]:
# Function to convert sentences into vectors using a trained model
def sentence_to_vector(model, sentence):
    vector = []
    for word in sentence:
        if word in model.wv:
            vector.append(model.wv[word])
    if len(vector) == 0:
        vector.append(np.zeros(model.vector_size))
    return np.mean(vector, axis=0)

# Convert sentences into vectors using CBOW model
X_train_cbow = np.array([sentence_to_vector(cbow_model, sentence) for sentence in X_train])
X_test_cbow = np.array([sentence_to_vector(cbow_model, sentence) for sentence in X_test])

# Convert sentences into vectors using Skip-gram model
X_train_skipgram = np.array([sentence_to_vector(skipgram_model, sentence) for sentence in X_train])
X_test_skipgram = np.array([sentence_to_vector(skipgram_model, sentence) for sentence in X_test])


In [ ]:

# Define NN model architecture
model = Sequential()
#model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
 
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(100,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model

In [ ]:
X_test[0]

In [ ]:
X_train_cbow[0]

In [ ]:
y_test

In [ ]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
# Train the model using CBOW vectors
model.fit(X_train_cbow, y_train, epochs=20, batch_size=16, validation_data=(X_test_cbow, y_test))


In [ ]:
#cbow_embedding_matrix = generate_word_embeddings(X_train, X_test, model_type='cbow')
#cbow_model = create_model(cbow_embedding_matrix)
#cbow_model.fit(X_train_pad, y_train, epochs=10, batch_size=16, validation_data=(X_test_pad, y_test), verbose=0)
cbow_pred = (model.predict(X_train_cbow) > 0.5).astype("int32")


In [ ]:
cbow_accuracy = accuracy_score(y_test, cbow_pred)
cbow_precision = precision_score(y_test, cbow_pred)
cbow_recall = recall_score(y_test, cbow_pred)
cbow_f1 = f1_score(y_test, cbow_pred)

In [ ]:
# Train the model using Skip-gram vectors
model.fit(X_train_skipgram, y_train, epochs=10, batch_size=16, validation_data=(X_test_skipgram, y_test))
